In [1]:
!pip install dm-haiku numpyro

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 371.0/371.0 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.7/312.7 kB 11.6 MB/s eta 0:00:00


In [8]:
from typing import Sequence, NamedTuple
import copy, time, json

import numpy as np
import plotly.express as px

import numpyro
import numpyro.distributions as dist
from numpyro.infer import MCMC, HMC, NUTS

import jax
import jax.numpy as jnp
import jax.tree_util as jtree
import haiku as hk
import optax

In [3]:
class SGLDConfig(NamedTuple):
  epsilon: float
  gamma: float
  num_steps: int

def generate_rngkey_tree(key_or_seed, tree_or_treedef):
    rngseq = hk.PRNGSequence(key_or_seed)
    return jtree.tree_map(lambda _: next(rngseq), tree_or_treedef)

def optim_sgld(epsilon, rngkey_or_seed, preconditioning=None):
    @jax.jit
    def sgld_delta(g, rngkey, p=1):
        eta = jax.random.normal(rngkey, shape=g.shape) * jnp.sqrt(epsilon * p)
        return -epsilon * p * g / 2 + eta

    def init_fn(_):
        return rngkey_or_seed

    if preconditioning is None:
        update_map = lambda grads, rngkey_tree: jax.tree_map(sgld_delta, grads, rngkey_tree)
    else:
        update_map = lambda grads, rngkey_tree: jax.tree_map(sgld_delta, grads, rngkey_tree, preconditioning)

    @jax.jit
    def update_fn(grads, state):
        rngkey, new_rngkey = jax.random.split(state)
        rngkey_tree = generate_rngkey_tree(rngkey, grads)
        updates = update_map(grads, rngkey_tree)
        return updates, new_rngkey
    return optax.GradientTransformation(init_fn, update_fn)


def create_local_logposterior(avgnegloglikelihood_fn, num_training_data, w_init, gamma, itemp):
    def helper(x, y):
        return jnp.sum((x - y)**2)

    def _logprior_fn(w):
        sqnorm = jax.tree_util.tree_map(helper, w, w_init)
        return jax.tree_util.tree_reduce(lambda a,b: a + b, sqnorm)

    def logprob(w, x, y):
        loglike = -num_training_data * avgnegloglikelihood_fn(w, x, y)
        logprior = -gamma / 2 * _logprior_fn(w)
        return itemp * loglike + logprior
    return logprob


In [4]:
def mala_acceptance_probability(current_point, proposed_point, loss_and_grad_fn, step_size):
    """
    Calculate the acceptance probability for a MALA transition.

    Args:
    current_point: The current point in parameter space.
    proposed_point: The proposed point in parameter space.
    loss_and_grad_fn (function): Function to compute loss and loss gradient at a point.
    step_size (float): Step size parameter for MALA.

    Returns:
    float: Acceptance probability for the proposed transition.
    """
    # Compute the gradient of the loss at the current point
    current_loss, current_grad = loss_and_grad_fn(current_point)
    proposed_loss, proposed_grad = loss_and_grad_fn(proposed_point)

    # Compute the log of the proposal probabilities (using the Gaussian proposal distribution)
    log_q_proposed_to_current = -jnp.sum((current_point - proposed_point - (step_size * 0.5 * -proposed_grad)) ** 2) / (2 * step_size)
    log_q_current_to_proposed = -jnp.sum((proposed_point - current_point - (step_size * 0.5 * -current_grad)) ** 2) / (2 * step_size)

    # Compute the acceptance probability
    acceptance_log_prob = log_q_proposed_to_current - log_q_current_to_proposed + current_loss - proposed_loss
    return jnp.minimum(1.0, jnp.exp(acceptance_log_prob))

# Example usage:
# Define a simple log likelihood function for demonstration
def log_likelihood_example(x):
    return -0.5 * jnp.sum(x**2)

loss_and_grad_fn = jax.jit(jax.value_and_grad(lambda x: -log_likelihood_example(x), argnums=0))

# Current and proposed points for demonstration
current_point = jnp.array([1.0, 2.0])
proposed_point = jnp.array([1.5, 2.5])
step_size = 0.1

# Calculate the acceptance probability
acceptance_prob = mala_acceptance_probability(current_point, proposed_point, loss_and_grad_fn, step_size)
acceptance_prob


Array(0.95719343, dtype=float32)

In [5]:
def pack_params(params):
    params_flat, treedef = jax.tree_util.tree_flatten(params)
    shapes = [p.shape for p in params_flat]
    indices = np.cumsum([p.size for p in params_flat])
    params_packed = jnp.concatenate([jnp.ravel(p) for p in params_flat])
    pack_info = (treedef, shapes, indices)
    return params_packed, pack_info

def unpack_params(params_packed, pack_info):
    treedef, shapes, indices = pack_info
    params_split = jnp.split(params_packed, indices)
    params_flat = [jnp.reshape(p, shape) for p, shape in zip(params_split, shapes)]
    params = jax.tree_util.tree_unflatten(treedef, params_flat)
    return params

# MLPs

In [10]:
class ReluNetwork(hk.Module):
    def __init__(self, layer_widths):
        super().__init__()
        self.layer_widths = layer_widths

    def __call__(self, x):
        for width in self.layer_widths[:-1]:
            x = hk.Linear(width)(x)
            x = jax.nn.relu(x)
        x = hk.Linear(self.layer_widths[-1])(x)
        return x

# Function to initialize and apply the DLN model
def forward_fn(x, layer_widths):
    net = ReluNetwork(layer_widths)
    return net(x)

# Create a Haiku-transformed version of the model
def create_model(layer_widths):
    model = hk.without_apply_rng(hk.transform(lambda x: forward_fn(x, layer_widths)))
    return model

def generate_training_data(true_param, model, input_dim, num_samples):
    inputs = np.random.uniform(-10, 10, size=(num_samples, input_dim))

    # Apply the true model to generate outputs
    true_outputs = model.apply(true_param, inputs)

    return inputs, true_outputs

def mse_loss(param, model, inputs, targets):
    predictions = model.apply(param, inputs)
    return jnp.mean((predictions - targets) ** 2)

def create_minibatches(inputs, targets, batch_size, num_iter=None, shuffle=True):
    assert len(inputs) == len(targets)

    if num_iter is None:
        num_iter = len(inputs)

    if shuffle:
        indices = np.random.permutation(len(inputs))
    else:
        indices = np.arange(len(inputs))

    iter = 0
    while True:
        for start_idx in range(0, len(inputs) - batch_size + 1, batch_size):
            excerpt = indices[start_idx:start_idx + batch_size]
            yield inputs[excerpt], targets[excerpt]

            iter += 1
            if iter >= num_iter:
                return



In [12]:
input_dim = 10  # Dimension of the input
layer_widths = [20, 20, 10]

# Create the model
model = create_model(layer_widths)

# Initialize the "true" model parameters
rngkey = jax.random.PRNGKey(0)
dummy_input = jnp.zeros((1, input_dim), dtype=jnp.float32)
rngkey, subkey = jax.random.split(rngkey)
true_param = model.init(rngkey, dummy_input)

# Set biases to not be zero
for i in range(len(layer_widths)-1):
    rngkey, subkey = jax.random.split(rngkey)
    loc = f'relu_network/linear_{i}' if i > 0 else 'relu_network/linear'
    true_param[loc]['b'] = jax.random.normal(subkey, (layer_widths[i],))


In [23]:
step_size = 1e-5
gamma = 1.0
batch_size = 32
num_samples = 10000
num_runs = 5

# Sweep over different training data size
dataset_sizes = np.round(10**np.linspace(2, 5, 7)).astype(np.int64)
sgld_times = []
mala_times = []
memories = []
sgld_lambdas = []
mala_lambdas = []
for num_training_data in dataset_sizes:
    sgld_times.append([])
    mala_times.append([])
    memories.append([])
    sgld_lambdas.append([])
    mala_lambdas.append([])

    for run in range(num_runs):
        print(f"Training set size: {num_training_data}, run {run}")

        # Generate training data
        np.random.seed(0)
        x_train, y_train = generate_training_data(true_param, model, input_dim, num_training_data)
        print(f"x_train shape: {x_train.shape}, y_train shape: {y_train.shape}")
        jtree.tree_map_with_path(lambda path, x: x.shape, true_param)

        memory = (x_train.size * x_train.itemsize) + (y_train.size * y_train.itemsize)
        memories[-1].append(memory)
        print("Training data memory: ", memory)

        itemp = 1 / np.log(num_training_data)
        param_init = copy.deepcopy(true_param)

        loss_fn = jax.jit(lambda param, inputs, targets: mse_loss(param, model, inputs, targets))
        local_logprob = create_local_logposterior(
                avgnegloglikelihood_fn=loss_fn,
                num_training_data=num_training_data,
                w_init=param_init,
                gamma=gamma,
                itemp=itemp,
            )

        # Run SGLD
        sgld_start_time = time.time()

        sgld_grad_fn = jax.jit(jax.value_and_grad(lambda w, x, y: -local_logprob(w, x, y), argnums=0))

        rngkey = jax.random.PRNGKey(run)
        sgldoptim = optim_sgld(step_size, rngkey)
        samples = []
        losses = []
        accept_probs = []
        opt_state = sgldoptim.init(param_init)
        param = param_init
        t = 0
        for x_batch, y_batch in create_minibatches(x_train, y_train, batch_size=batch_size, num_iter=num_samples):
            old_param = param.copy()

            nll, grads = sgld_grad_fn(param, x_batch, y_batch)
            updates, opt_state = sgldoptim.update(grads, opt_state)
            param = optax.apply_updates(param, updates)
            samples.append(param)
            losses.append(loss_fn(param, x_batch, y_batch))

            t += 1

            if t % 20 == 0:
                old_param_packed, pack_info = pack_params(old_param)
                param_packed, _ = pack_params(param)
                def grad_fn_packed(w):
                    nll, grad = sgld_grad_fn(unpack_params(w, pack_info), x_batch, y_batch)
                    grad_packed, _ = pack_params(grad)
                    return nll, grad_packed
                accept_probs.append(mala_acceptance_probability(
                    old_param_packed, param_packed, grad_fn_packed, step_size))

        init_loss = loss_fn(param_init, x_train, y_train)
        lambdahat = ((np.mean(losses) - init_loss) * num_training_data * itemp).item()

        sgld_time = time.time() - sgld_start_time

        sgld_lambdas[-1].append(lambdahat)
        sgld_times[-1].append(sgld_time)

        print("SGLD lambda: ", lambdahat)
        print("SGLD accept prob: ", np.mean(accept_probs))
        print("SGLD execution time: ", sgld_time)

        # Run MALA
        mala_start_time = time.time()

        rngkey = jax.random.PRNGKey(run)

        param_init = copy.deepcopy(true_param)
        beta = itemp*num_training_data
        mala_step_size = np.sqrt(step_size)
        num_steps = 1

        loss_fn = jax.jit(lambda w: mse_loss(w, model, x_train, y_train))
        potential_fn = jax.jit(lambda w: -local_logprob(w, x_train, y_train))

        hmc_kernel = HMC(
            #potential_fn=lambda param: beta * loss_fn(param),
            potential_fn=potential_fn,
            step_size=mala_step_size,
            trajectory_length=mala_step_size*num_steps,
            adapt_step_size=False,
            adapt_mass_matrix=False
        )
        mcmc = MCMC(hmc_kernel, num_samples=10000, num_warmup=0)
        rngkey, subkey = jax.random.split(rngkey)
        mcmc.run(subkey, init_params=param_init)

        # Extract samples
        samples = mcmc.get_samples()

        init_loss = loss_fn(param_init)
        losses = (jax.lax.map(loss_fn, samples))
        lambdahat = ((np.mean(losses) - init_loss) * beta).item()

        mala_time = time.time() - mala_start_time

        mala_lambdas[-1].append(lambdahat)
        mala_times[-1].append(mala_time)

        print("MALA lambda: ", lambdahat)
        print("MALA execution time: ", mala_time)

# Save results to JSON
results = {
    "dataset_sizes": dataset_sizes.tolist(),
    "SGLD": {
        "lambdas": sgld_lambdas,
        "times": sgld_times
    },
    "MALA": {
        "lambdas": mala_lambdas,
        "times": mala_times
    },
}
json.dump(results, open("mala_v_sgld_results.json", "w"))


Training set size: 100, run 0
x_train shape: (100, 10), y_train shape: (100, 10)
Training data memory:  12000
SGLD lambda:  57.37425994873047
SGLD accept prob:  0.99507684
SGLD execution time:  33.13397693634033


sample: 100%|██████████| 10000/10000 [05:20<00:00, 31.23it/s, 1 steps of size 3.16e-03. acc. prob=1.00]


MALA lambda:  50.44807434082031
MALA execution time:  322.0246331691742
Training set size: 100, run 1
x_train shape: (100, 10), y_train shape: (100, 10)
Training data memory:  12000
SGLD lambda:  57.60322952270508
SGLD accept prob:  0.9949707
SGLD execution time:  31.66882586479187


sample: 100%|██████████| 10000/10000 [05:16<00:00, 31.60it/s, 1 steps of size 3.16e-03. acc. prob=1.00]


MALA lambda:  56.770263671875
MALA execution time:  317.98200130462646
Training set size: 100, run 2
x_train shape: (100, 10), y_train shape: (100, 10)
Training data memory:  12000
SGLD lambda:  55.59379577636719
SGLD accept prob:  0.99578637
SGLD execution time:  32.193074226379395


...

MALA lambda:  347.5590515136719
MALA execution time:  1049.0516211986542
Training set size: 100000, run 3
x_train shape: (100000, 10), y_train shape: (100000, 10)
Training data memory:  12000000
SGLD lambda:  350.6711120605469
SGLD accept prob:  0.9890411
SGLD execution time:  33.73149871826172


sample: 100%|██████████| 10000/10000 [16:15<00:00, 10.25it/s, 1 steps of size 3.16e-03. acc. prob=0.26]


MALA lambda:  355.29193115234375
MALA execution time:  1088.828677892685
Training set size: 100000, run 4
x_train shape: (100000, 10), y_train shape: (100000, 10)
Training data memory:  12000000
SGLD lambda:  349.2572021484375
SGLD accept prob:  0.9918579
SGLD execution time:  33.226011514663696


sample: 100%|██████████| 10000/10000 [15:46<00:00, 10.57it/s, 1 steps of size 3.16e-03. acc. prob=0.20]


MALA lambda:  345.3625183105469
MALA execution time:  1058.6680824756622
